In [8]:
from binance import Client

In [9]:
import pandas as pd

In [10]:
from datetime import datetime

In [11]:
from binance import BinanceSocketManager

In [12]:
import time

In [13]:
import numpy as np

In [14]:
import sys

In [15]:
from decimal import Decimal, ROUND_HALF_UP

In [16]:
%run ./Binance_keys.ipynb

In [17]:
client = Client(api_key, api_secret)

In [18]:
xcoin = 'AVAXUSDT'
scoin = 'USDT'
crypto = 'AVAX'

In [19]:
def getminutedata(symbol, interval, lookback): # we order the data in dataframe using pandas
    frame = pd.DataFrame(client.get_historical_klines(symbol, interval, lookback+' min ago UTC'))
    frame = frame.iloc[:,:6]
    frame.columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume']
    frame = frame.set_index('Time')
    frame.index = pd.to_datetime(frame.index, unit= 'ms')
    frame = frame.astype(float)
    return frame

In [29]:
def currentprice(symbol): # we get the last price from market for the specified ticker
    dic = client.get_ticker(symbol= xcoin)
    price = dic.get("lastPrice")
    strings = [str(integer) for integer in price]
    a_string = "".join(strings)
    a_string = a_string.split('.', 1)[0]+ '.'+a_string.split('.', 2)[1] 
    a_float = float(a_string)
    return a_float

In [32]:
def getlastsell(): # we get the price from last sell
    frame = pd.DataFrame(client.get_my_trades(symbol=xcoin))
    frame = frame.iloc[1:, -9:-1]
    frame = frame[(frame['isBuyer']==False)]
    frame = frame.iloc[-1:, :2]
    frame = frame.astype(float)
    frame = frame.set_index('qty')
    #frame = frame.get(key='price')
    dic = frame.to_dict("list")
    listdic = dic.get('price')
    strings = [str(integer) for integer in listdic]
    a_string = "".join(strings)
    a_string = a_string.split('.', 1)[0]+ '.'+a_string.split('.', 2)[1]
    a_float = float(a_string)
    return a_float

In [34]:
def getlastbuy(): # we get the price from last buyout
    frame = pd.DataFrame(client.get_my_trades(symbol=xcoin))
    frame = frame.iloc[1:, -9:-1]
    frame = frame[(frame['isBuyer']==True)]
    frame = frame.iloc[-1:, :2]
    frame = frame.astype(float)
    #frame = frame.get(key='price')
    frame = frame.set_index('qty')
    dic = frame.to_dict("list")
    listdic = dic.get('price')
    strings = [str(integer) for integer in listdic]
    a_string = "".join(strings)
    a_string = a_string.split('.', 1)[0]+ '.'+a_string.split('.', 2)[1]
    a_float = float(a_string)
    return a_float

In [36]:
def getlastbuyqty(): # we get how much we bought from last time 
    frame = pd.DataFrame(client.get_my_trades(symbol=xcoin))
    frame = frame.iloc[1:, -9:-1]
    frame = frame[(frame['isBuyer']==True)]
    frame = frame.iloc[-1:, :2]
    frame = frame.astype(float)
    #frame = frame.get(key='price')
    frame = frame.set_index('price')
    dic = frame.to_dict("list")
    listdic = dic.get('qty')
    strings = [str(integer) for integer in listdic]
    a_string = "".join(strings)
    a_string = a_string.split('.', 1)[0]+ '.'+a_string.split('.', 2)[1]
    a_float = float(a_string)
    return a_float

In [16]:
def to_money(amount) -> Decimal: #this is important to use, since money usually has 2 decimals
    money = Decimal(amount).quantize(Decimal('.00'), rounding=ROUND_HALF_UP)
    return money

In [17]:
def strategyCrypto(symbol, bought = False):
    while True:
        try:
        # by default when you start the program, it will start with bought in false, and wait to buy until the
        #cumulative return has decreased more than 0.05%, remember our period of time is the last 10 minutes
            if not bought:
                while True:
                    time.sleep(.25)
                    framing = getminutedata(symbol, '1m', '10m') #we order the data with interval of 1 minute, for the last 10 minutes
                    actualprice = currentprice(symbol)
                    lastsell = getlastsell()
                    cumulret = (framing.Open.pct_change() +1).cumprod() -1
                    stablecoin = client.get_asset_balance(asset=scoin)
                    stablecoinstr = stablecoin.get("free")
                    stablecoinfloat = float(stablecoinstr)
                    stablecoinavail = stablecoinfloat * 1
                    numbercrypto = stablecoinavail // actualprice
                    if cumulret[-1] < -0.00539:
                        order = client.create_order(symbol = symbol, side='BUY', type= 'MARKET', quantity = numbercrypto)
                        print(order)
                        bought = True
                        break
                    else:
                        pass
                    
            # once we have bought, we need to wait to sell until the actual price is greater than last time buyout
            # AND cumulative return has increased 15% in the last 8 hours or increased 0.05% in the last 10 minutes
            
            if bought:
                while True:
                    time.sleep(.25)
                    framing = getminutedata(symbol, '1m', '30m')
                    framingUrgent = getminutedata(symbol, '1m', '480m')
                    framingnormal = getminutedata(symbol, '1m', '10m')
                    urgentret = (framingUrgent.Open.pct_change() +1).cumprod() -1
                    normalret = (framingnormal.Open.pct_change() +1).cumprod() -1
                    actualprice = currentprice(symbol)
                    ticker = client.get_asset_balance(asset=crypto)
                    tickerstr = ticker.get("free")
                    tickerfloat = float(tickerstr)
                    tickeravail = tickerfloat * 0.47
                    tickerurgent = tickerfloat * 0.955
                    tickeravail = to_money(tickeravail)
                    tickerurgent = to_money(tickerurgent)
                    lastbuy = getlastbuy()
                    lastbuyqty = getlastbuyqty()
                    lastbuyqty = to_money(lastbuyqty)
                    sincebuy = framing.loc[framing.index > pd.to_datetime(order['transactTime'], unit = 'ms')]
                    
                    
                    lastbuyqtyfinal = order.get('executedQty')
                    stringsqty = [str(integer) for integer in lastbuyqtyfinal]
                    qty_string = "".join(stringsqty)
                    qty_string = qty_string.split('.', 1)[0]+ '.'+qty_string.split('.', 2)[1]
                    qty_float = float(qty_string)
                    qty_float_r = qty_float * 0.955
                    lastbuyqtyfinalavail = to_money(qty_float_r)
    
                    
                    if len(sincebuy) > 0:
                        sincebuyret = (sincebuy.Open.pct_change() +1).cumprod() -1
                        #sincebuyret[-1] > 0.0009 and 
                        if (((actualprice*.997) > (lastbuy*1.001)) and urgentret[-1] > .15):
                            order = client.create_order(symbol = symbol, side='SELL', type= 'MARKET', quantity = tickerurgent)
                            print(order)
                            bought = False
                            amount_to_transfer_partial = ((getlastsell() - getlastbuy()) * float(tickerurgent))
                            #amount_to_transfer = float(to_money(amount_to_transfer_partial)) * 0.82
                            amount_to_transfer = amount_to_transfer_partial * 0.82
                            amount_to_transfer_money = to_money(amount_to_transfer)
                            #once we get a profit, we transfer the profit to another place, example to the margin wallet
                            transfer_profit = client.transfer_spot_to_margin(asset=scoin, amount= amount_to_transfer_money)
                            # if for some reason the crypto has increased 15% the cumulative return in the last 8 hours, then wait
                            # 90 minutes
                            time.sleep(5400)
                            break
                        #elif (actualprice*.997) > (lastbuy*1.001):
                        elif (((actualprice*.997) > (lastbuy*1.001)) and normalret[-1] > .00539):
                            order = client.create_order(symbol = symbol, side='SELL', type= 'MARKET', quantity = lastbuyqtyfinalavail)
                            print(order)
                            bought = False
                            amount_to_transfer_partial = ((getlastsell() - getlastbuy()) * float(lastbuyqtyfinalavail))
                            amount_to_transfer = amount_to_transfer_partial * 0.82
                            amount_to_transfer_money = to_money(amount_to_transfer)
                            #once we get a profit, we transfer the profit to another place, example to the margin wallet
                            transfer_profit = client.transfer_spot_to_margin(asset=scoin, amount= amount_to_transfer_money)
                            break
        except: # catch *all* exceptions
            e = sys.exc_info()[0]
            i = sys.exc_info()[1]
            o = sys.exc_info()[2]
            print("Oops!  That was invalid data.  Error detected at...")
            now = datetime.now()
            print("Date:", now.strftime("%Y-%m-%d %H:%M:%S"))
            print(e)
            print(i)
            print(o)

In [ ]:
strategyCrypto(xcoin)

{'symbol': 'AVAXUSDT', 'orderId': 1978967401, 'orderListId': -1, 'clientOrderId': 'ndpcQBMoj0wre6lMF7SDqL', 'transactTime': 1663828563013, 'price': '0.00000000', 'origQty': '50.00000000', 'executedQty': '50.00000000', 'cummulativeQuoteQty': '842.50000000', 'status': 'FILLED', 'timeInForce': 'GTC', 'type': 'MARKET', 'side': 'BUY', 'fills': [{'price': '16.85000000', 'qty': '50.00000000', 'commission': '0.05000000', 'commissionAsset': 'AVAX', 'tradeId': 199498162}]}
{'symbol': 'AVAXUSDT', 'orderId': 1979016428, 'orderListId': -1, 'clientOrderId': 'qhjFCfoXTLRQVITVB08ub7', 'transactTime': 1663832824454, 'price': '0.00000000', 'origQty': '47.75000000', 'executedQty': '47.75000000', 'cummulativeQuoteQty': '816.04750000', 'status': 'FILLED', 'timeInForce': 'GTC', 'type': 'MARKET', 'side': 'SELL', 'fills': [{'price': '17.09000000', 'qty': '47.75000000', 'commission': '0.81604750', 'commissionAsset': 'USDT', 'tradeId': 199501900}]}
{'symbol': 'AVAXUSDT', 'orderId': 1979124722, 'orderListId': -1

{'symbol': 'AVAXUSDT', 'orderId': 1979972917, 'orderListId': -1, 'clientOrderId': 'WP0CwiqeKDjryd3Fdjkcy6', 'transactTime': 1663886763620, 'price': '0.00000000', 'origQty': '44.00000000', 'executedQty': '44.00000000', 'cummulativeQuoteQty': '776.60000000', 'status': 'FILLED', 'timeInForce': 'GTC', 'type': 'MARKET', 'side': 'BUY', 'fills': [{'price': '17.65000000', 'qty': '44.00000000', 'commission': '0.04400000', 'commissionAsset': 'AVAX', 'tradeId': 199565445}]}
{'symbol': 'AVAXUSDT', 'orderId': 1979985701, 'orderListId': -1, 'clientOrderId': 'Z94vTgtZAnCxB1NuUBRvKN', 'transactTime': 1663888023598, 'price': '0.00000000', 'origQty': '42.02000000', 'executedQty': '42.02000000', 'cummulativeQuoteQty': '745.85500000', 'status': 'FILLED', 'timeInForce': 'GTC', 'type': 'MARKET', 'side': 'SELL', 'fills': [{'price': '17.75000000', 'qty': '42.02000000', 'commission': '0.74585500', 'commissionAsset': 'USDT', 'tradeId': 199566018}]}
{'symbol': 'AVAXUSDT', 'orderId': 1980001010, 'orderListId': -1

Oops!  That was invalid data.  Error detected at...
Date: 2022-09-23 01:21:27
<class 'binance.exceptions.BinanceAPIException'>
APIError(code=-2010): Account has insufficient balance for requested action.
Oops!  That was invalid data.  Error detected at...
Date: 2022-09-23 01:21:31
<class 'binance.exceptions.BinanceAPIException'>
APIError(code=-2010): Account has insufficient balance for requested action.
Oops!  That was invalid data.  Error detected at...
Date: 2022-09-23 01:21:35
<class 'binance.exceptions.BinanceAPIException'>
APIError(code=-2010): Account has insufficient balance for requested action.
Oops!  That was invalid data.  Error detected at...
Date: 2022-09-23 01:21:39
<class 'binance.exceptions.BinanceAPIException'>
APIError(code=-2010): Account has insufficient balance for requested action.
Oops!  That was invalid data.  Error detected at...
Date: 2022-09-23 01:21:46
<class 'binance.exceptions.BinanceAPIException'>
APIError(code=-2010): Account has insufficient balance fo

Oops!  That was invalid data.  Error detected at...
Date: 2022-09-23 01:23:52
<class 'binance.exceptions.BinanceAPIException'>
APIError(code=-2010): Account has insufficient balance for requested action.
Oops!  That was invalid data.  Error detected at...
Date: 2022-09-23 01:23:56
<class 'binance.exceptions.BinanceAPIException'>
APIError(code=-2010): Account has insufficient balance for requested action.
Oops!  That was invalid data.  Error detected at...
Date: 2022-09-23 01:24:00
<class 'binance.exceptions.BinanceAPIException'>
APIError(code=-2010): Account has insufficient balance for requested action.
Oops!  That was invalid data.  Error detected at...
Date: 2022-09-23 01:24:04
<class 'binance.exceptions.BinanceAPIException'>
APIError(code=-2010): Account has insufficient balance for requested action.
Oops!  That was invalid data.  Error detected at...
Date: 2022-09-23 01:24:07
<class 'binance.exceptions.BinanceAPIException'>
APIError(code=-2010): Account has insufficient balance fo

Oops!  That was invalid data.  Error detected at...
Date: 2022-09-23 02:10:03
<class 'binance.exceptions.BinanceAPIException'>
APIError(code=-2010): Account has insufficient balance for requested action.
Oops!  That was invalid data.  Error detected at...
Date: 2022-09-23 02:10:07
<class 'binance.exceptions.BinanceAPIException'>
APIError(code=-2010): Account has insufficient balance for requested action.
Oops!  That was invalid data.  Error detected at...
Date: 2022-09-23 02:10:10
<class 'binance.exceptions.BinanceAPIException'>
APIError(code=-2010): Account has insufficient balance for requested action.
Oops!  That was invalid data.  Error detected at...
Date: 2022-09-23 02:10:13
<class 'binance.exceptions.BinanceAPIException'>
APIError(code=-2010): Account has insufficient balance for requested action.
Oops!  That was invalid data.  Error detected at...
Date: 2022-09-23 02:10:17
<class 'binance.exceptions.BinanceAPIException'>
APIError(code=-2010): Account has insufficient balance fo

...

Oops!  That was invalid data.  Error detected at...
Date: 2022-09-27 09:23:09
<class 'binance.exceptions.BinanceAPIException'>
APIError(code=-2010): Account has insufficient balance for requested action.
Oops!  That was invalid data.  Error detected at...
Date: 2022-09-27 09:23:13
<class 'binance.exceptions.BinanceAPIException'>
APIError(code=-2010): Account has insufficient balance for requested action.
Oops!  That was invalid data.  Error detected at...
Date: 2022-09-27 09:23:16
<class 'binance.exceptions.BinanceAPIException'>
APIError(code=-2010): Account has insufficient balance for requested action.
Oops!  That was invalid data.  Error detected at...
Date: 2022-09-27 09:23:21
<class 'binance.exceptions.BinanceAPIException'>
APIError(code=-2010): Account has insufficient balance for requested action.
Oops!  That was invalid data.  Error detected at...
Date: 2022-09-27 09:23:25
<class 'binance.exceptions.BinanceAPIException'>
APIError(code=-2010): Account has insufficient balance fo

Oops!  That was invalid data.  Error detected at...
Date: 2022-09-27 09:25:24
<class 'binance.exceptions.BinanceAPIException'>
APIError(code=-2010): Account has insufficient balance for requested action.
Oops!  That was invalid data.  Error detected at...
Date: 2022-09-27 09:25:27
<class 'binance.exceptions.BinanceAPIException'>
APIError(code=-2010): Account has insufficient balance for requested action.
Oops!  That was invalid data.  Error detected at...
Date: 2022-09-27 09:25:31
<class 'binance.exceptions.BinanceAPIException'>
APIError(code=-2010): Account has insufficient balance for requested action.
Oops!  That was invalid data.  Error detected at...
Date: 2022-09-27 09:25:35
<class 'binance.exceptions.BinanceAPIException'>
APIError(code=-2010): Account has insufficient balance for requested action.
Oops!  That was invalid data.  Error detected at...
Date: 2022-09-27 09:25:38
<class 'binance.exceptions.BinanceAPIException'>
APIError(code=-2010): Account has insufficient balance fo

Oops!  That was invalid data.  Error detected at...
Date: 2022-09-27 09:27:36
<class 'binance.exceptions.BinanceAPIException'>
APIError(code=-2010): Account has insufficient balance for requested action.
Oops!  That was invalid data.  Error detected at...
Date: 2022-09-27 09:27:40
<class 'binance.exceptions.BinanceAPIException'>
APIError(code=-2010): Account has insufficient balance for requested action.
Oops!  That was invalid data.  Error detected at...
Date: 2022-09-27 09:27:43
<class 'binance.exceptions.BinanceAPIException'>
APIError(code=-2010): Account has insufficient balance for requested action.
Oops!  That was invalid data.  Error detected at...
Date: 2022-09-27 09:27:47
<class 'binance.exceptions.BinanceAPIException'>
APIError(code=-2010): Account has insufficient balance for requested action.
Oops!  That was invalid data.  Error detected at...
Date: 2022-09-27 09:27:51
<class 'binance.exceptions.BinanceAPIException'>
APIError(code=-2010): Account has insufficient balance fo

Oops!  That was invalid data.  Error detected at...
Date: 2022-09-27 09:35:52
<class 'binance.exceptions.BinanceAPIException'>
APIError(code=-2010): Account has insufficient balance for requested action.
Oops!  That was invalid data.  Error detected at...
Date: 2022-09-27 09:35:56
<class 'binance.exceptions.BinanceAPIException'>
APIError(code=-2010): Account has insufficient balance for requested action.
Oops!  That was invalid data.  Error detected at...
Date: 2022-09-27 09:36:01
<class 'binance.exceptions.BinanceAPIException'>
APIError(code=-2010): Account has insufficient balance for requested action.
Oops!  That was invalid data.  Error detected at...
Date: 2022-09-27 09:36:11
<class 'binance.exceptions.BinanceAPIException'>
APIError(code=-2010): Account has insufficient balance for requested action.
Oops!  That was invalid data.  Error detected at...
Date: 2022-09-27 09:36:21
<class 'binance.exceptions.BinanceAPIException'>
APIError(code=-2010): Account has insufficient balance fo

Oops!  That was invalid data.  Error detected at...
Date: 2022-09-27 09:40:54
<class 'binance.exceptions.BinanceAPIException'>
APIError(code=-2010): Account has insufficient balance for requested action.
Oops!  That was invalid data.  Error detected at...
Date: 2022-09-27 09:41:00
<class 'binance.exceptions.BinanceAPIException'>
APIError(code=-2010): Account has insufficient balance for requested action.
Oops!  That was invalid data.  Error detected at...
Date: 2022-09-27 09:41:07
<class 'binance.exceptions.BinanceAPIException'>
APIError(code=-2010): Account has insufficient balance for requested action.
Oops!  That was invalid data.  Error detected at...
Date: 2022-09-27 09:41:13
<class 'binance.exceptions.BinanceAPIException'>
APIError(code=-2010): Account has insufficient balance for requested action.
Oops!  That was invalid data.  Error detected at...
Date: 2022-09-27 09:41:19
<class 'binance.exceptions.BinanceAPIException'>
APIError(code=-2010): Account has insufficient balance fo

# 